In [2]:
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import tensorflow as tf

import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore"

if (dir := str(Path(os.getcwd()).parent)) not in sys.path:
    sys.path.append(dir)

from mylib.metrics import Metrics, CMetrics

SEED = 73
np.random.seed = SEED

In [3]:
wine = pd.read_csv('../data/wine_preprocessed.csv', sep=';')
rain = pd.read_csv('../data/rain_preprocessed.csv', sep=';')

In [4]:
X_wine = wine.drop(['quality'], axis=1)
y_wine = wine['quality']
print(X_wine.shape, y_wine.shape)
X_wine_train, X_wine_test, y_wine_train, y_wine_test = train_test_split(X_wine, y_wine, test_size=0.33)

rain0 = rain[rain['RainTomorrow'] == 0]
rain1 = rain[rain['RainTomorrow'] == 1]
rain0 = resample(rain0,
                 replace=False,
                 n_samples=len(rain1),
                 random_state=73)
rain = pd.concat([rain1, rain0])
X_rain = rain.drop(columns=['RainTomorrow'])
y_rain = rain['RainTomorrow']
print(X_rain.shape, y_rain.shape)
X_rain_train, X_rain_test, y_rain_train, y_rain_test = train_test_split(X_rain, y_rain, test_size=0.33)

(6497, 12) (6497,)
(63754, 23) (63754,)


In [5]:
print_r_metrics = lambda true, pred: Metrics(true, pred)()
print_c_metrics = lambda true, pred: CMetrics(true, pred)()

In [49]:
from mylib.perceptron import Perceptron
from mylib.layer.layer import Layer
from mylib.optimizer.SGD import SGD
from mylib.optimizer.momentum import Momentum
from mylib.optimizer.RMSProp import RMSProp
from mylib.optimizer.adam import Adam

In [32]:
pc = (
    Perceptron([
        Layer(16, 'relu', 23),
        Layer(16, 'sigmoid'),
        Layer(2, 'softmax'),
    ])
    .compile(loss='mse')
    .fit(np.array(X_rain_train), np.array(y_rain_train), epochs=8, batch_size=32)
)
pc_pred = pc.predict(np.array(X_rain_test))
print(pc_pred)
print_c_metrics(y_rain_test, np.argmax(pc_pred, axis=1))

[[0.49997156 0.50002844]
 [0.50003557 0.49996443]
 [0.50002434 0.49997566]
 ...
 [0.50006176 0.49993824]
 [0.499996   0.500004  ]
 [0.49995681 0.50004319]]
	precision	recall	f1-score
0	     0.63	  0.75	    0.68
1	     0.69	  0.56	    0.62
accuracy			    0.65



In [92]:
adam = (
    Perceptron([
        Layer(16, 'tanh', 23),
        Layer(8, 'tanh'),
        Layer(2, 'softmax'),
    ])
    .compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    .fit(np.array(X_rain_train), np.array(y_rain_train), epochs=15, batch_size=128)
)
adam_pred = adam.predict(np.array(X_rain_test))
print_c_metrics(y_rain_test, np.argmax(adam_pred, axis=1))

	precision	recall	f1-score
0	     0.58	  0.72	    0.64
1	     0.63	  0.47	    0.54
accuracy			    0.60



In [116]:
a = np.array(
    [
        [
            [1, 1],
            [2, 2],
            [3, 3],
        ],
        [
            [4, 4],
            [5, 5],
            [6, 6],
        ],
    ]
)

print(a.shape)
print(a)
# a = np.array(
#     [
#         [1, 1],
#         [2, 2]
#     ]
# )


(2, 3, 2)
[[[1 1]
  [2 2]
  [3 3]]

 [[4 4]
  [5 5]
  [6 6]]]


In [117]:
np.reshape(a, [-1, np.prod(a.shape[1:])])

array([[1, 1, 2, 2, 3, 3],
       [4, 4, 5, 5, 6, 6]])

In [126]:
x = np.random.randn(5, 3)
xx = np.reshape(x, [-1, np.prod(x.shape[1:])])
xx

array([[ 0.60183296,  0.68811257, -0.50832032],
       [ 0.86401266, -0.18630604,  1.37179407],
       [ 0.43507748,  0.51400761,  0.54007806],
       [-1.34794607,  0.6353066 ,  0.73903983],
       [-0.64729701,  0.08304134, -0.19947973]])